In [ ]:
import numpy as np
import pandas as pd
from debugpy.launcher import channel

import matplotlib.pyplot as plt
import seaborn as sns
import os
#os.chdir('/Users/admin/PycharmProjects/ET_eksploracja/hyperscanning-signal-analysis')
%matplotlib inline

In [ ]:
!pwd

In [ ]:
import sys
sys.path.append('../src')
import dataloader

# Functions to process position, pupil diameter, annotstions and blinks from ET are now avaliable in dataloader.py

In [ ]:
def reset_time_to_first_event(df):
    '''
    Reset the time column in the dataframe to start from the first event timestamp.
    We assume the events are one of the movie watching or talking parts of the experiment.
    :param df: dataframe with 'time' and 'event' columns
    :return: dataframe with reset time
    '''
    df = df.loc[df['event'].notnull()]
    first = df['time'].iloc[0]
    df['time'] = df['time'] - first
    return df

# Tests

## Hyperparametrs setting

In [ ]:
Fs = 1024
order = 351 # order of the filter
cutoff = 1 # cutoff frequency of the filter in Hz   

## Loading ET data from watching movies

In [ ]:
ch_pos_df_0 = pd.read_csv('../data/et_data/000/ch_gaze_positions_on_surface_Surface 1.csv')
cg_pos_df_0 = pd.read_csv('../data/et_data/000/cg_gaze_positions_on_surface_Surface 1.csv')
ch_pupil_df_0 = pd.read_csv('../data/et_data/000/ch_pupil_positions.csv')
cg_pupil_df_0 = pd.read_csv('../data/et_data/000/cg_pupil_positions.csv')
annotations_0 = pd.read_csv('../data/et_data/000/annotations.csv')
cg_blinks_0 = pd.read_csv('../data/et_data/000/cg_blinks.csv')
ch_blinks_0 = pd.read_csv('../data/et_data/000/ch_blinks.csv')

## Loading ET data from talk 1

In [ ]:
ch_pupil_df_1 = pd.read_csv('../data/et_data/001/ch_pupil_positions.csv')
cg_pupil_df_1 = pd.read_csv('../data/et_data/001/cg_pupil_positions.csv')
annotations_1 = pd.read_csv('../data/et_data/001/annotations.csv')
cg_blinks_1 = pd.read_csv('../data/et_data/001/cg_blinks.csv')
ch_blinks_1 = pd.read_csv('../data/et_data/001/ch_blinks.csv')

## Loading ET data from talk 2

In [ ]:
ch_pupil_df_2 = pd.read_csv('../data/et_data/002/ch_pupil_positions.csv')
cg_pupil_df_2 = pd.read_csv('../data/et_data/002/cg_pupil_positions.csv')
annotations_2 = pd.read_csv('../data/et_data/002/annotations.csv')
cg_blinks_2 = pd.read_csv('../data/et_data/002/cg_blinks.csv')
ch_blinks_2 = pd.read_csv('../data/et_data/002/ch_blinks.csv')

## Construct data frame for ET data

In [ ]:
df = pd.DataFrame()

In [ ]:
df['time'] = dataloader.process_time_et(ch_pos_df_0, cg_pos_df_0, ch_pupil_df_0, cg_pupil_df_0, ch_pupil_df_1, cg_pupil_df_1, ch_pupil_df_2, cg_pupil_df_2, Fs=Fs)

In [ ]:
df['ch_x_interp'],df['ch_y_interp'] = dataloader.process_pos(ch_pos_df_0, df['time'])
df['cg_x_interp'],df['cg_y_interp'] = dataloader.process_pos(cg_pos_df_0, df['time'])

In [ ]:
df['ch_diameter3d'] = dataloader.process_pupil(ch_pupil_df_0, df,'ch')
df['ch_diameter3d'] = dataloader.process_pupil(ch_pupil_df_1, df,'ch')
df['ch_diameter3d'] = dataloader.process_pupil(ch_pupil_df_2, df,'ch')

df['cg_diameter3d'] = dataloader.process_pupil(cg_pupil_df_0, df,'cg')
df['cg_diameter3d'] = dataloader.process_pupil(cg_pupil_df_1, df,'cg')
df['cg_diameter3d'] = dataloader.process_pupil(cg_pupil_df_2, df,'cg')

In [ ]:
df['event'] = dataloader.process_event_et(annotations_0, df)
df['event'] = dataloader.process_event_et(annotations_1, df, 'talk1')
df['event'] = dataloader.process_event_et(annotations_2, df, 'talk2')
print(df['event'].unique())

In [ ]:
df['cg_blinks'] = dataloader.process_blinks(cg_blinks_0, df,'cg')
df['cg_blinks'] = dataloader.process_blinks(cg_blinks_1, df,'cg')
df['cg_blinks'] = dataloader.process_blinks(cg_blinks_2, df,'cg')

df['ch_blinks'] = dataloader.process_blinks(ch_blinks_0, df,'ch')
df['ch_blinks'] = dataloader.process_blinks(ch_blinks_1, df,'ch')
df['ch_blinks'] = dataloader.process_blinks(ch_blinks_2, df,'ch')

In [ ]:
#df['dist'] = np.sqrt((df['ch_x_interp'] - df['cg_x_interp'])**2 + (df['ch_y_interp'] - df['cg_y_interp'])**2)

In [ ]:
df.info()

In [ ]:
# df['ch_blinks'].value_counts()

## Connecting the ET with EEG

## Load EEG data

In [ ]:

# creates a DataLoader class object that creates a structure described in the docs folder (data_structure_spec.md) from raw data
# data = DataLoader(, )
multimodal_data = dataloader.load_eeg_data("W_010", "../data/eeg_data/", False)


Convert data from multimodal data to dataframe

In [ ]:
channel_mapping = multimodal_data.eeg_channel_mapping
x = multimodal_data.eeg_data[channel_mapping['Fp1'],:]
d = multimodal_data.diode

In [ ]:
time = np.arange(0, len(x)/multimodal_data.eeg_fs, 1/multimodal_data.eeg_fs)
plt.figure(figsize=(10,6))
plt.plot(time, x)
plt.show()

In [ ]:
df = reset_time_to_first_event(df)

In [ ]:
df.loc[df['event'] == 'talk1', 'time']

In [ ]:
multimodal_data.events

In [ ]:
minimum = min(t['start'] for t in multimodal_data.events if 'start' in t)
minimum

In [ ]:
minimum = int(minimum * multimodal_data.eeg_fs)

In [ ]:
df['time']

In [ ]:
time = np.arange(0, len(x)/multimodal_data.eeg_fs, 1/multimodal_data.eeg_fs)
plt.figure(figsize=(10,6))
plt.plot(time[minimum:], x[minimum:])
plt.plot(time[minimum:], d[minimum:]*200)
plt.show()